In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=10)
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [2]:
train=pd.read_csv("data/train.csv", index_col="PassengerId")

print(train.shape)
train.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test=pd.read_csv("data/test.csv", index_col="PassengerId")

print(test.shape)
test.head()

(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Preprocessing

### Encoding sex

In [4]:
# train[""].value_counts()
train["Sex_encode"]=train["Sex"].replace("male",0).replace("female",1)
print(train["Sex_encode"].shape)

train[["Sex_encode"]].head()

(891,)


,Sex_encode
PassengerId,
1,0
2,1
3,1
4,1
5,0


In [5]:
# train[""].value_counts()
test["Sex_encode"]=test["Sex"].replace("male",0).replace("female",1)
print(test["Sex_encode"].shape)

test[["Sex_encode"]].head()

(418,)


,Sex_encode
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [6]:
pd.pivot_table(data=train, index="SibSp",values="Survived")

,Survived
SibSp,
0,0.345395
1,0.535885
2,0.464286
3,0.250000
4,0.166667
5,0.000000
8,0.000000


In [7]:
pd.pivot_table(data=train, index="Parch",values="Survived")

,Survived
Parch,
0,0.343658
1,0.550847
2,0.500000
3,0.600000
4,0.000000
5,0.200000
6,0.000000


In [8]:
pd.pivot_table(data=train, index=["Parch","SibSp"],values="Survived")

Survived
Parch SibSp          
0     0      0.303538
      1      0.520325
      2      0.250000
      3      1.000000
1     0      0.657895
      1      0.596491
      2      0.857143
      3      0.000000
      4      0.000000
2     0      0.724138
      1      0.631579
      2      0.500000
      3      0.285714
      4      0.333333
      5      0.000000
      8      0.000000
3     0      1.000000
      1      0.333333
      2      1.000000
4     0      0.000000
      1      0.000000
5     0      0.000000
      1      0.333333
6     1      0.000000

In [9]:
train["Sibpar"]=train["SibSp"].isnull()
train["Sibpar"].value_counts()

False    891
Name: Sibpar, dtype: int64

In [10]:
# Si ==0 (1,3), ==1(0:2),==2(0:2),==3(0,2)-> 이후 올 다이
train["Sibpar"]
train.loc[(train["SibSp"]==0) & (train["Parch"]==1),"Sibpar"]=True
train.loc[(train["SibSp"]==0) & (train["Parch"]==3),"Sibpar"]=True

train.loc[(train["SibSp"]==1) & (train["Parch"]==0),"Sibpar"]=True
train.loc[(train["SibSp"]==1) & (train["Parch"]==1),"Sibpar"]=True
train.loc[(train["SibSp"]==1) & (train["Parch"]==2),"Sibpar"]=True

train.loc[(train["SibSp"]==2) & (train["Parch"]==0),"Sibpar"]=True
train.loc[(train["SibSp"]==2) & (train["Parch"]==1),"Sibpar"]=True
train.loc[(train["SibSp"]==2) & (train["Parch"]==2),"Sibpar"]=True

train.loc[(train["SibSp"]==3) & (train["Parch"]==0),"Sibpar"]=True
train.loc[(train["SibSp"]==3) & (train["Parch"]==2),"Sibpar"]=True

In [11]:
train["Sibpar"].value_counts()
print(train["Sibpar"].shape)
print(train.shape)

(891,)
(891, 13)


In [12]:
train[["SibSp","Parch","Sibpar"]].head(30)

,SibSp,Parch,Sibpar
PassengerId,,,
1,1,0,True
2,1,0,True
3,0,0,False
4,1,0,True
5,0,0,False
6,0,0,False
7,0,0,False
8,3,1,False
9,0,2,False


In [13]:
test["Sibpar"]=test["SibSp"].isnull()
test["Sibpar"].value_counts()
test["Sibpar"].head(30)

PassengerId
892    False
893    False
894    False
895    False
896    False
897    False
898    False
899    False
900    False
901    False
902    False
903    False
904    False
905    False
906    False
907    False
908    False
909    False
910    False
911    False
912    False
913    False
914    False
915    False
916    False
917    False
918    False
919    False
920    False
921    False
Name: Sibpar, dtype: bool

In [14]:
#test.loc[(test["SibSp"]==0) & (test["Parch"]==1),"Sibpar"]=True
#test.loc[(test["SibSp"]==0) & (test["Parch"]==3),"Sibpar"]=True

#test.loc[(test["SibSp"]==1) & (test["Parch"]==0),"Sibpar"]=True
#test.loc[(test["SibSp"]==1) & (test["Parch"]==1),"Sibpar"]=True
#test.loc[(test["SibSp"]==1) & (test["Parch"]==2),"Sibpar"]=True

#test.loc[(test["SibSp"]==2) & (test["Parch"]==0),"Sibpar"]=True
#test.loc[(test["SibSp"]==2) & (test["Parch"]==1),"Sibpar"]=True
#test.loc[(test["SibSp"]==2) & (test["Parch"]==2),"Sibpar"]=True

#test.loc[(test["SibSp"]==3) & (test["Parch"]==0),"Sibpar"]=True
#test.loc[(test["SibSp"]==3) & (test["Parch"]==2),"Sibpar"]=True

In [15]:
# Si ==0 (1,3), ==1(0:2),==2(0:2),==3(0,2)-> 이후 올 다이
test.loc[(test["SibSp"]==0) & (test["Parch"]==1|3),"Sibpar"]=True
test.loc[(test["SibSp"]==1) & (test["Parch"]<3),"Sibpar"]=True
test.loc[(test["SibSp"]==2) & (test["Parch"]<3),"Sibpar"]=True
test.loc[(test["SibSp"]==3) & (test["Parch"]==0|2),"Sibpar"]=True


In [16]:
test[["SibSp","Parch","Sibpar"]].head(30)

,SibSp,Parch,Sibpar
PassengerId,,,
892,0,0,False
893,1,0,True
894,0,0,False
895,0,0,False
896,1,1,True
897,0,0,False
898,0,0,False
899,1,1,True
900,0,0,False


In [17]:
print(test["Sibpar"].shape)
print(test.shape)

(418,)
(418, 12)


In [18]:
test[["SibSp","Parch","Sibpar"]].head(20)

,SibSp,Parch,Sibpar
PassengerId,,,
892,0,0,False
893,1,0,True
894,0,0,False
895,0,0,False
896,1,1,True
897,0,0,False
898,0,0,False
899,1,1,True
900,0,0,False


In [19]:
train["Embarked_C"]=train["Embarked"]=="C"
train["Embarked_S"]=train["Embarked"]=="S"
train["Embarked_Q"]=train["Embarked"]=="Q"

print(train.shape)

train[["Embarked","Embarked_C","Embarked_S","Embarked_Q"]].head()

(891, 16)


,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
1,S,False,True,False
2,C,True,False,False
3,S,False,True,False
4,S,False,True,False
5,S,False,True,False


In [20]:
test["Embarked_C"]=test["Embarked"]=="C"
test["Embarked_S"]=test["Embarked"]=="S"
test["Embarked_Q"]=test["Embarked"]=="Q"

print(test.shape)

test[["Embarked","Embarked_C","Embarked_S","Embarked_Q"]].head()

(418, 15)


,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
892,Q,False,False,True
893,S,False,True,False
894,Q,False,False,True
895,S,False,True,False
896,S,False,True,False


In [33]:
agesur=pd.pivot_table(data=train, index="Age",values="Survived")
print(agesur.shape)
agesur.head()

(88, 1)


,Survived
Age,
0.42,1.0
0.67,1.0
0.75,1.0
0.83,1.0
0.92,1.0


In [34]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Sibpar,Embarked_C,Embarked_S,Embarked_Q,Fare(fill)
PassengerId,,,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,True,False,True,False,7.2500
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,True,True,False,False,71.2833
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,False,False,True,False,7.9250
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,True,False,True,False,53.1000
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,False,False,True,False,8.0500


### Encoding Age as group

In [47]:
train["Age(fill)_young"]=train["Age"]<=15
train["Age(fill)_middle"]=(train["Age"]<=30)&(train["Age"]>15)
train["Age(fill)_old"]=train["Age"]>30
print(train.shape)
train["Age(fill)"].head()

(891, 21)


PassengerId
1    False
2     True
3    False
4     True
5     True
Name: Age(fill), dtype: bool

In [48]:
train[["Age","Age(fill)_young","Age(fill)_middle","Age(fill)_old"]]

,Age,Age(fill)_young,Age(fill)_middle,Age(fill)_old
PassengerId,,,,
1,22.0,False,True,False
2,38.0,False,False,True
3,26.0,False,True,False
4,35.0,False,False,True
5,35.0,False,False,True
...,...,...,...,...
887,27.0,False,True,False
888,19.0,False,True,False
889,NaN,False,False,False


In [49]:
test["Age(fill)_young"]=test["Age"]<=15
test["Age(fill)_middle"]=(test["Age"]<=30)&(test["Age"]>15)
test["Age(fill)_old"]=test["Age"]>30
print(test.shape)
test["Age(fill)"].head()

(418, 20)


PassengerId
892     True
893     True
894     True
895    False
896    False
Name: Age(fill), dtype: bool

### missing values in file 

In [21]:
train['Fare(fill)'] =train['Fare'].fillna(0)
train[['Fare(fill)']].head(30)
print(train.shape)

(891, 17)


In [22]:
test['Fare(fill)'] =test['Fare'].fillna(0)
print(test.shape)

test[['Fare(fill)']].head(30)

(418, 16)


,Fare(fill)
PassengerId,
892,7.8292
893,7.0000
894,9.6875
895,8.6625
896,12.2875
897,9.2250
898,7.6292
899,29.0000
900,7.2292


In [23]:
test[test['Fare(fill)'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Sibpar,Embarked_C,Embarked_S,Embarked_Q,Fare(fill)
PassengerId,,,,,,,,,,,,,,,,


In [24]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Sibpar,Embarked_C,Embarked_S,Embarked_Q,Fare(fill)
PassengerId,,,,,,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,False,False,False,True,7.8292
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,True,False,True,False,7.0000
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,False,False,False,True,9.6875
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,False,False,True,False,8.6625
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1,True,False,True,False,12.2875


In [50]:
feature_names=["Pclass","Sex_encode","Fare(fill)","Age(fill)_young","Age(fill)_middle","Age(fill)_old","Embarked_C","Embarked_S","Embarked_Q","Sibpar"]
feature_names

['Pclass',
 'Sex_encode',
 'Fare(fill)',
 'Age(fill)_young',
 'Age(fill)_middle',
 'Age(fill)_old',
 'Embarked_C',
 'Embarked_S',
 'Embarked_Q',
 'Sibpar']

In [51]:
X_train=train[feature_names]

print(X_train.shape)
X_train.head()

(891, 10)


,Pclass,Sex_encode,Fare(fill),Age(fill)_young,Age(fill)_middle,Age(fill)_old,Embarked_C,Embarked_S,Embarked_Q,Sibpar
PassengerId,,,,,,,,,,
1,3,0,7.2500,False,True,False,False,True,False,True
2,1,1,71.2833,False,False,True,True,False,False,True
3,3,1,7.9250,False,True,False,False,True,False,False
4,1,1,53.1000,False,False,True,False,True,False,True
5,3,0,8.0500,False,False,True,False,True,False,False


In [52]:
X_test=test[feature_names]

print(X_test.shape)
X_test.head()

(418, 10)


,Pclass,Sex_encode,Fare(fill),Age(fill)_young,Age(fill)_middle,Age(fill)_old,Embarked_C,Embarked_S,Embarked_Q,Sibpar
PassengerId,,,,,,,,,,
892,3,0,7.8292,False,False,True,False,False,True,False
893,3,1,7.0000,False,False,True,False,True,False,True
894,2,0,9.6875,False,False,True,False,False,True,False
895,3,0,8.6625,False,True,False,False,True,False,False
896,3,1,12.2875,False,True,False,False,True,False,True


In [53]:
label_name="Survived"
y_train=train[label_name]

print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [54]:
model.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [55]:
prediction=model.predict(X_test)
print(prediction.shape)
prediction[0:5]

(418,)


array([0, 0, 0, 0, 1])

In [56]:
gender=pd.read_csv("data/gender_submission.csv",index_col="PassengerId")
print(gender.shape)
gender.head()

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [57]:
gender["Survived"]=prediction
gender.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [58]:
gender.to_csv("data/renew.csv")